# calc_U_h_coeffs.ipynb

This script implements heat transfer correlations based in boundary layers to estimate heat transfer coefficients for natural convection at the tank bottom and tank sidewalls. Correlations for the Nusselt number over different interfaces are presented as a function of the characteristic length, $l_0$, of convection:

- $h_b$: Heat transfer coefficient by Natural Convection at the tank bottom, using $l_o = d_o$ where $l_o$ is the characteristic length.
- $h_i$: Heat transfer coefficient by Natural Convection between the liquid ammonia and the internal tank wall, using $l_o = H$.
- $h_{o,nc}$: Heat transfer coefficient by Natural Convection between the air and the external tank wall, using $l_o = H$.
- $h_{o,fc}$: Heat transfer coefficient by Forced Convection between the air and the external tank wall, using $l_o = \pi d_o /(2H+2d_o)$.

Source: 


In [1]:
# Import modules
import CoolProp.CoolProp as CP
import numpy as np

# CONSTANT PARAMETERS
fluid = 'Ammonia'  # Fluid
Air = 'Air' 
pressure = 101325  # Operating pressure 1 atm in Pa

# LNG tank properties
Q_roof = 0 # Roof heat ingress / W
T_air = 5.3286+273.15 # Temperature of the environment K
V_tank = 88023.6952 # Tank volume / m^3
a    = 0.5  # aspect ratio H/D
d_i  = ((4 * V_tank)/(np.pi * a))**(1/3) # internal diameter / m
e    = 10.6972*2 - 10.24*2 # twice the wall thickness / m
d_o  = d_i + e   # External diameter / m

H = a*d_i  # Height in meters
g = 9.81      # Acceleration due to gravity in m/s^2
T_air = 5.3286+273.15 # Ambient air temperature / K, San Gregorio, Punta Arenas, Chile
P_air = 101325 # / Pa
fill_rate = 1 # Fill rate of the tank, from 0 to 1
# Tank properties

e_p = 18*0.0254 # Perlite insulation layer thickness / m
k_p = 0.0411    # Perlite thermal conductivity Wm^-1K^-1

# Stainless steel dopado con Mn
e_m = 1*0.0254 # Metalic layer insulation layer thickness / m
k_m = 32.5 # Metalic layer thermal conductivity Wm^-1K^-1

# Start of iteration
delta_T = 0.1  # Temperature difference in K

# Get the saturation temperature at 1 atm
T_sat = CP.PropsSI('T', 'P', pressure, 'Q', 0, fluid)# + 1e-3

# We assess thermophysical properties at film temperature
T_f = T_sat + delta_T/2

# Assumption: LNH3 is not boiling at the tank bottom
# so the properties of the liquid are calculated
# assuming saturation AT THE FILM TEMPERATURE
rho = CP.PropsSI('D', 'Q', 0, 'T', T_f, fluid)
rho_air = CP.PropsSI('D','P',P_air,'T', T_air, Air)

# Get the derivative of density with respect to temperature at constant pressure
# drho_dT = CP.PropsSI('d(D)/d(T)|P', 'P', pressure, 'Q', 0, fluid)
drho_dT = CP.PropsSI('d(D)/d(T)|P', 'T', T_f, 'Q', 0, fluid)
drho_dT_air = CP.PropsSI('d(D)/d(T)|P', 'T', T_air, 'P', P_air, Air)

# Calculate the thermal expansion coefficient
beta = - drho_dT / rho
beta_air = - drho_dT_air/rho_air

# Get the Prandtl number at saturation temperature and 1 atm
Pr = CP.PropsSI('Prandtl', 'T', T_f, 'Q', 0, fluid)
Pr_air = CP.PropsSI('Prandtl', 'T', T_air, 'P', P_air, Air) 

# Get the dynamic viscosity at saturation temperature and 1 atm
mu = CP.PropsSI('V', 'T', T_f, 'Q', 0, fluid)
mu_air = CP.PropsSI('V','T', T_air, 'P', P_air, Air)

# Calculate the kinematic viscosity
nu = mu / rho
nu_air = mu_air/rho_air

# Get the thermal conductivity at saturation temperature and 1 atm
k = CP.PropsSI('L', 'T', T_f, 'Q', 0, fluid)
k_air = CP.PropsSI('L','T', T_air, 'P', P_air, Air)

# Calculate the thermal diffusivity
alpha = k / (rho * CP.PropsSI('C', 'T', T_f, 'Q', 0, fluid))
alpha_air = k_air/(rho_air * CP.PropsSI('C', 'T', T_air, 'P', P_air, Air))
# Calculate the Rayleigh number
Ra = (g * beta * delta_T * H**3) / (nu * alpha)
Ra_air = (g * beta_air * delta_T * H**3) / (nu_air * alpha_air)

print(f'Thermal expansion coefficient of liquid ammonia at saturation temperature and 1 atm: {beta:.6e} 1/K')
print(f'Prandtl number of ammonia: {Pr:.6f}')
print(f'Rayleigh number of ammonia: {Ra:.6e}')
print(f'Thermal expansion coefficient of air at atmospheric conditions: {beta_air:.6e} 1/K')
print(f'Prandtl number of air: {Pr_air:.6f}')
print(f'Rayleigh number or air: {Ra_air:.6e}')

Thermal expansion coefficient of liquid ammonia at saturation temperature and 1 atm: 1.807695e-03 1/K
Prandtl number of ammonia: 1.709651
Rayleigh number of ammonia: 6.095147e+14
Thermal expansion coefficient of air at atmospheric conditions: 3.602948e-03 1/K
Prandtl number of air: 0.710027
Rayleigh number or air: 3.699634e+11


## Functions definition

In [2]:
# Calculates internal heat transfer coefficient at the base of a cylindrical tank
def h_i_base(Pr, Ra, k, D):
    f1 = (1 + (0.492*Pr)**(9/16))**(-16/9)
    print(f'Raf1: {Ra*f1:.6e}')
    # Heat emission at lower surface 
    Nu_b = 0.6 * (Ra * f1)**(1/5)
    print(f'Nusselt number: {Nu_b:.3e}')
    h_b = Nu_b * k / D 
    print(f'Internal heat transfer coefficient at the tank base, h_b: {h_b:.3e}', 'Wm^-2K^-1')
    return h_b

def U_base(h_b, r_i, r_o, k_p):
    '''
    This equation does not include the heat transfer from the outside
    Inputs:
        h_b: heat transfer coefficient from the bottom
        r_i: tank internal radius / m
        r_2: metallic layer radius / m
        k_m: metallic layer thermal conductivity / Wm^-1K^-1
        r_o: insulation olayer radius / m
        k_p: insulation layer thermal conductivity / Wm^-1K^-1

    The resistance of the inner metallic wall is neglected
    '''
    U = (r_o/(r_i*h_b) + r_o * np.log(r_o/r_i)/k_p)**(-1)
    return U

def h_i_sides(Pr, Ra, k, H, D):
    f1 = (1+(0.492/Pr)**(9/16))**(-16/9)
    Nu_plate = (0.825+0.387*(Ra*f1)**(1/6))**2
    Nu = Nu_plate + 0.97*(H/D)
    print(f'Ra number: {Ra:.3e}')
    print(f'Nusselt number: {Nu:.3e}')

    # The characteristic length is the height of the tank for vertical
    # natural convection
    h_sides = Nu * k / H 

    print(f'Internal heat transfer coefficient at the tank sides, h_side: {h_sides:.3e}', 'Wm^-2K^-1')
    return h_sides

def h_forced_sides(Pr, Re, k, d):
    Nu_lam = 0.664 * Re**(1/2) * Pr**(1/3)
    Nu_turb = (0.037 * Re**(0.8) * Pr)/(1 + 2.443 * Re**(-0.1) * (Pr**(2/3) - 1))
    Nu = 0.3 + (Nu_lam**2 + Nu_turb**2)**(1/2)

    # Streamed length
    # l_0 = np.pi * d / 2
    l_0 = (np.pi * d_o * H) / (2 * (H + d_o)) 
    h_forced = Nu * k / l_0
    print(f'External heat transfer coefficient at the tank sides, h_forced_sides: {h_forced:.3e}', 'Wm^-2K^-1')
    return h_forced

def U_sides(h_si, h_so, r_i, r_o, k_p):
    '''

    Overall heat transfer coefficient at the tank sides
    
    Inputs:
        h_si: internal heat transfer coefficient
        h_so: external heat transfer coefficient
        r_i: tank internal radius / m
        r_2: metallic layer radius / m
        k_m: metallic layer thermal conductivity / Wm^-1K^-1
        r_o: insulation layer radius / m
        k_p: insulation layer thermal conductivity / Wm^-1K^-1
    '''
    U = ( r_o/(r_i*h_si) + r_o * (np.log(r_o/r_i)/k_p ) + 1/(h_so))**(-1)
    return U


# Alternative correlation
def h_cylinder_crossflow(U, D, T_inf, T_s, P=101325.0):
    # film temperature
    T_f = 0.5*(T_inf + T_s)
    # air properties at film T and P
    rho = CP.PropsSI('D','P',P,'T',T_f,'Air')
    mu  = CP.PropsSI('VISCOSITY','P',P,'T',T_f,'Air')
    k   = CP.PropsSI('CONDUCTIVITY','P',P,'T',T_f,'Air')
    cp  = CP.PropsSI('CPMASS','P',P,'T',T_f,'Air')
    Pr  = cp*mu/k
    Re  = rho*U*D/mu

    # Churchill–Bernstein (valid for all Re, Pr)
    term = (0.62*Re**0.5*Pr**(1/3)) / (1 + (0.4/Pr)**(2/3))**0.25
    Nu   = 0.3 + term * (1 + (Re/282000.0)**(5/8))**(4/5)
    return Nu * k / D

## Local Heat Transfer Coefficients

In [3]:
h_so = h_i_sides(Pr_air, Ra_air, k_air, H, d_o) # External heat transfer coefficient

h_si = h_i_sides(Pr, Ra, k_air, fill_rate*H, d_i) # Internal heat transfer coefficient

Ra number: 3.700e+11
Nusselt number: 8.021e+02
Internal heat transfer coefficient at the tank sides, h_side: 6.540e-01 Wm^-2K^-1
Ra number: 6.095e+14
Nusselt number: 1.017e+04
Internal heat transfer coefficient at the tank sides, h_side: 8.291e+00 Wm^-2K^-1


$$ h(T_w - T_L) = U(T_{air} - T_L)$$

In [4]:
# Internal radius
r_i = d_i/2

# Radius to the end of the insulation layer
r_o = r_i + e_p

# Assumption: temperature of the outer wall is the same as the air temperature
h_b = h_i_base(Pr, Ra, k, d_i)

# Based on external area
U_b = U_base(h_b, r_i, r_o, k_p)

Raf1: 1.934060e+14
Nusselt number: 4.320e+02
Internal heat transfer coefficient at the tank base, h_b: 4.726e+00 Wm^-2K^-1


The correlation works for $$ 10^3 < Ra*f_1 <= 10^{10} $$

## Automated iteration for the bottom of the tank

In [5]:
err_dT = 1 # K
tol = 1e-4 # err
delta_T = 0.01 # K

while err_dT > tol:
    # Get the saturation temperature at 1 atm
    T_sat = CP.PropsSI('T', 'P', pressure, 'Q', 0, fluid)# + 1e-3

    # We assess thermophysical properties at film temperature
    T_f = T_sat + delta_T/2

    # Assumption: LNH3 is not boiling at the tank bottom
    # so the properties of the liquid are calculated
    # assuming saturation AT THE FILM TEMPERATURE
    rho = CP.PropsSI('D', 'Q', 0, 'T', T_f, fluid)

    # Get the derivative of density with respect to temperature at constant pressure
    drho_dT = CP.PropsSI('d(D)/d(T)|P', 'T', T_f, 'Q', 0, fluid)

    # Calculate the thermal expansion coefficient
    beta = - drho_dT / rho

    # Get the Prandtl number at saturation temperature and 1 atm
    Pr = CP.PropsSI('Prandtl', 'T', T_f, 'Q', 0, fluid)

    # Get the dynamic viscosity at saturation temperature and 1 atm
    mu = CP.PropsSI('V', 'T', T_f, 'Q', 0, fluid)

    # Calculate the kinematic viscosity
    nu = mu / rho

    # Get the thermal conductivity at saturation temperature and 1 atm
    k = CP.PropsSI('L', 'T', T_f, 'Q', 0, fluid)

    # Calculate the thermal diffusivity
    alpha = k / (rho * CP.PropsSI('C', 'T', T_f, 'Q', 0, fluid))

    # Calculate the Rayleigh number
    Ra = (g * beta * delta_T * H**3) / (nu * alpha)
    
    # Assumption: temperature of the outer wall is the same as the air temperature
    h_b = h_i_base(Pr, Ra, k, d_i)

    # Based on external area
    # U = U_base(h_b, r_i, r_o, k_p)
    U_b = U_base(h_b, r_i, r_o, k_p)
    
    delta_T_new = U_b/h_b * (T_air-T_sat)
    err_dT = abs(delta_T_new - delta_T)
    print("dT_new = %.3f K " % delta_T_new)
    print("dT_err", err_dT)
    delta_T = delta_T_new

Raf1: 1.931943e+13
Nusselt number: 2.725e+02
Internal heat transfer coefficient at the tank base, h_b: 2.982e+00 Wm^-2K^-1
dT_new = 1.122 K 
dT_err 1.1122542320741768
Raf1: 2.197630e+15
Nusselt number: 7.023e+02
Internal heat transfer coefficient at the tank base, h_b: 7.665e+00 Wm^-2K^-1
dT_new = 0.445 K 
dT_err 0.6776293624153734
Raf1: 8.635433e+14
Nusselt number: 5.827e+02
Internal heat transfer coefficient at the tank base, h_b: 6.369e+00 Wm^-2K^-1
dT_new = 0.534 K 
dT_err 0.08915855864336664
Raf1: 1.037830e+15
Nusselt number: 6.045e+02
Internal heat transfer coefficient at the tank base, h_b: 6.606e+00 Wm^-2K^-1
dT_new = 0.515 K 
dT_err 0.018896799900611105
Raf1: 1.000859e+15
Nusselt number: 6.001e+02
Internal heat transfer coefficient at the tank base, h_b: 6.559e+00 Wm^-2K^-1
dT_new = 0.519 K 
dT_err 0.0036737724038889974
Raf1: 1.008045e+15
Nusselt number: 6.010e+02
Internal heat transfer coefficient at the tank base, h_b: 6.568e+00 Wm^-2K^-1
dT_new = 0.518 K 
dT_err 0.000726715

In [6]:
print("U_b = %.5e W m^-2 K^-1 " % U_b)

U_b = 8.80129e-02 W m^-2 K^-1 


## Automated iteration for the sides of the tank

Liquid side

In [7]:
err_dT = 1
tol = 1e-8
delta_T_side_i = 1
h_so = h_i_sides(Pr_air, Ra_air, k_air, H, d_o) # External heat transfer coefficient

# Liquid phase iteration

while err_dT > tol:
    # Get the saturation temperature at 1 atm
    T_sat = CP.PropsSI('T', 'P', pressure, 'Q', 0, fluid)# + 1e-3

    # Evaluate thermophysical properties at film temperature
    T_f = T_sat + delta_T_side_i/2

    # Assumption: LNH3 is boiling at the tank bottom
    # so the properties of the liquid are calculated
    # assuming saturation AT THE FILM TEMPERATURE
    rho = CP.PropsSI('D', 'Q', 0, 'T', T_f, fluid)

    # Get the derivative of density with respect to temperature at constant pressure
    drho_dT = CP.PropsSI('d(D)/d(T)|P', 'T', T_f, 'Q', 0, fluid)

    # Calculate the thermal expansion coefficient
    beta = - drho_dT / rho

    # Get the Prandtl number at saturation temperature and 1 atm
    Pr = CP.PropsSI('Prandtl', 'T', T_f, 'Q', 0, fluid)

    # Get the dynamic viscosity at saturation temperature and 1 atm
    mu = CP.PropsSI('V', 'T', T_f, 'Q', 0, fluid)

    # Calculate the kinematic viscosity
    nu = mu / rho

    # Get the thermal conductivity at saturation temperature and 1 atm
    k = CP.PropsSI('L', 'T', T_f, 'Q', 0, fluid)

    # Calculate the thermal diffusivity
    alpha = k / (rho * CP.PropsSI('C', 'T', T_f, 'Q', 0, fluid))

    # Calculate the Rayleigh number
    Ra = (g * beta * delta_T_side_i * H**3) / (nu * alpha)

#    print(f'Thermal expansion coefficient of liquid ammonia at saturation temperature and 1 atm: {beta:.6e} 1/K')
#    print(f'Prandtl number: {Pr:.6f}')
#    print(f'Rayleigh number: {Ra:.6e}')
    
    # Assumption: temperature of the outer wall is the same as the air temperature
    h_si = h_i_sides(Pr, Ra, k, fill_rate*H, d_i) # Internal heat transfer coefficient

    # Based on external area
    # U_sides(h_si, h_so, r_i, r_o, k_p)
    U_s = U_sides(h_si, h_so, r_i, r_o, k_p)
    
    delta_T_new = U_s/(h_si * d_o/d_i) * (T_air-T_sat)
    err_dT = abs(delta_T_new - delta_T_side_i)
    print("dT_err", err_dT)
    delta_T_side_i = delta_T_new



Ra number: 3.700e+11
Nusselt number: 8.021e+02
Internal heat transfer coefficient at the tank sides, h_side: 6.540e-01 Wm^-2K^-1
Ra number: 6.152e+15
Nusselt number: 2.185e+04
Internal heat transfer coefficient at the tank sides, h_side: 4.770e+02 Wm^-2K^-1
dT_err 0.9937343566657919
Ra number: 3.815e+13
Nusselt number: 4.076e+03
Internal heat transfer coefficient at the tank sides, h_side: 8.921e+01 Wm^-2K^-1
dT_err 0.027209974552440733
Ra number: 2.039e+14
Nusselt number: 7.081e+03
Internal heat transfer coefficient at the tank sides, h_side: 1.550e+02 Wm^-2K^-1
dT_err 0.014197643506261664
Ra number: 1.174e+14
Nusselt number: 5.903e+03
Internal heat transfer coefficient at the tank sides, h_side: 1.292e+02 Wm^-2K^-1
dT_err 0.0038470815346600754
Ra number: 1.408e+14
Nusselt number: 6.268e+03
Internal heat transfer coefficient at the tank sides, h_side: 1.372e+02 Wm^-2K^-1
dT_err 0.0013460580885890916
Ra number: 1.326e+14
Nusselt number: 6.145e+03
Internal heat transfer coefficient at t

Air

In [8]:
err_dT = 1
tol = 1e-8
delta_T_side_o = 0.1
# Air  iteration

while err_dT > tol:
    # Evaluate thermophysical properties at film temperature
    T_f = T_air + delta_T_side_o/2

    # Air density
    rho_air = CP.PropsSI('D','P', P_air,'T', T_f, Air)

    # Get the derivative of density with respect to temperature at constant pressure
    drho_dT = CP.PropsSI('d(D)/d(T)|P', 'P',P_air, 'T', T_f, Air)

    # Calculate the thermal expansion coefficient
    beta_air = - drho_dT / rho_air

    # Get the Prandtl number at saturation temperature and 1 atm
    Pr_air = CP.PropsSI('Prandtl', 'P',P_air, 'T', T_f, Air)


    # Get the dynamic viscosity at saturation temperature and 1 atm
    mu_air = CP.PropsSI('V', 'P',P_air, 'T', T_f, Air)

    # Calculate the kinematic viscosity
    nu_air = mu_air / rho_air

    # Get the thermal conductivity at saturation temperature and 1 atm
    k_air = CP.PropsSI('L', 'P',P_air, 'T', T_f, Air)


    # Calculate the thermal diffusivity
    alpha_air = k_air / (rho_air * CP.PropsSI('C', 'P',P_air, 'T', T_f, Air))

    # Calculate the Rayleigh number
    Ra_air = (g * beta_air * delta_T_side_o * H**3) / (nu_air * alpha_air)
    print("Ra_air = %.3e" % Ra_air)
    
    # Calculate new external heat transfer coefficient
    h_so = h_i_sides(Pr_air, Ra_air, k_air, H, d_o) # Internal heat transfer coefficient

    # Based on external area
    U_s = U_sides(h_si, h_so, r_i, r_o, k_p)
    # U_sides(h_si, h_so, r_i, r_o, k_p)
    
    delta_T_new = U_s/(h_so) * (T_air-T_sat)
    err_dT = abs(delta_T_new - delta_T_side_o)
    print("dT_err", err_dT)
    delta_T_side_o = delta_T_new



Ra_air = 3.697e+11
Ra number: 3.697e+11
Nusselt number: 8.019e+02
Internal heat transfer coefficient at the tank sides, h_side: 6.540e-01 Wm^-2K^-1
dT_err 4.536850070800132
Ra_air = 1.651e+13
Ra number: 1.651e+13
Nusselt number: 2.766e+03
Internal heat transfer coefficient at the tank sides, h_side: 2.272e+00 Wm^-2K^-1
dT_err 3.177254551213397
Ra_air = 5.335e+12
Ra number: 5.335e+12
Nusselt number: 1.911e+03
Internal heat transfer coefficient at the tank sides, h_side: 1.562e+00 Wm^-2K^-1
dT_err 0.6276728822847566
Ra_air = 7.589e+12
Ra number: 7.589e+12
Nusselt number: 2.144e+03
Internal heat transfer coefficient at the tank sides, h_side: 1.754e+00 Wm^-2K^-1
dT_err 0.21800415338554413
Ra_air = 6.809e+12
Ra number: 6.809e+12
Nusselt number: 2.070e+03
Internal heat transfer coefficient at the tank sides, h_side: 1.692e+00 Wm^-2K^-1
dT_err 0.06478998793417245
Ra_air = 7.041e+12
Ra number: 7.041e+12
Nusselt number: 2.092e+03
Internal heat transfer coefficient at the tank sides, h_side: 1.

In [11]:
print(U_s) # Global Heat Transfer coefficient from the sides of the tank
print(U_b) # Global Heat Transfer coefficient from the bottom

print(f'Global heat transfer coefficient at the tank base, U_b: {U_b:.3e}', 'Wm^-2K^-1')
print(f'Global heat transfer coefficient at the tank sides, U_s: {U_s:.3e}', 'Wm^-2K^-1')

0.08474016439105413
0.08801288511609139
Global heat transfer coefficient at the tank base, U_b: 8.801e-02 Wm^-2K^-1
Global heat transfer coefficient at the tank sides, U_s: 8.474e-02 Wm^-2K^-1


In [12]:
print(h_so)
print(h_b)
print(h_si)

1.7068554816740045
6.566663413712898
135.1452373183017


Forced convection

In [14]:
v_air = 8.056 # m s^-1
# Characteristic length is the streamwise length
l_0 = (np.pi * d_o * H) / (2 * (H + d_o)) 

Re = rho_air * v_air * l_0 / mu_air
h_forced = h_forced_sides(Pr_air, Re, k_air, d_o)

U_FC = U_sides(h_si, h_forced, r_i, r_o, k_p)
print("h_so_fc = %.4e W m^-2 K ^-1 " % h_forced)
print("U_FC| VDI Heat Atlas  = %.5e W m^-2 K ^-1 " % U_FC)

h_so_fcx = h_cylinder_crossflow(v_air, d_o, T_air, T_air, P_air)
U_FC = U_sides(h_si, h_so_fcx, r_i, r_o, k_p)
print("h_so_fc = %.4e W m^-2 K ^-1 " % h_so_fcx)
print("U_FC | cross flow = %.4e W m^-2 K ^-1 " % U_FC)

External heat transfer coefficient at the tank sides, h_forced_sides: 1.484e+01 Wm^-2K^-1
h_so_fc = 1.4839e+01 W m^-2 K ^-1 
U_FC| VDI Heat Atlas  = 8.86344e-02 W m^-2 K ^-1 
h_so_fc = 1.3745e+01 W m^-2 K ^-1 
U_FC | cross flow = 8.8592e-02 W m^-2 K ^-1 


Option 2 - Churchill-Bernstein